In [ ]:
import yaml
import sys
import os
import numpy as np
import torch 
os.chdir('/home/jonfrey/PLR3')
sys.path.insert(0, os.getcwd())
sys.path.append(os.path.join(os.getcwd() + '/src'))
sys.path.append(os.path.join(os.getcwd() + '/lib'))

import loaders_v2
from loaders_v2 import GenericDataset
from rotations import * 
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from visu import plot_pcd, Visualizer
import copy
from scipy.spatial.transform import Rotation as R
from helper import re_quat
from PIL import Image, ImageDraw
from deep_im import LossAddS
import copy
#from deep_im import flow_to_trafo
from visu import Visualizer
import matplotlib.pyplot as plt
import k3d
#exp_cfg_path = '/home/jonfrey/PLR3/yaml/exp/exp_ws_deepim_debug_natrix.yml'

env_cfg_path = '/home/jonfrey/PLR3/yaml/env/env_natrix_jonas.yml'
exp_cfg_path = '/home/jonfrey/PLR3/yaml/exp/exp_evaluate_pose_estimation.yml'
h = 480
w = 640
import k3d

def load_from_file(p):
    if os.path.isfile(p):
        with open(p, 'r') as f:
            data = yaml.safe_load(f)
    else:
        raise ValueError
    return data

exp = load_from_file(exp_cfg_path)
env = load_from_file(env_cfg_path)

dataset_train = GenericDataset(
    cfg_d=exp['d_train'],
    cfg_env=env)

batch = dataset_train[13450][0] #bann 10450
points, choose, img, target, model_points, idx = batch[0:6]
depth_img, label_img, img_orig, cam = batch[6:10]
gt_rot_wxyz, gt_trans, unique_desig = batch[10:13]

real_img, render_img, real_d, render_d, gt_label_cropped = batch[13:18]
pred_rot_wxyz, pred_trans, pred_points, h_render,h_real, render_img_original = batch[18:24]
u_map, v_map, flow_mask,  bb , depth_render_original= batch[24:]



                

In [ ]:
# Evaluate Ray Tracering 
# DepthMap -> Reproject 

In [ ]:
# Plotting Helper Functions (not needed)
class Drawer():
    def __init__(self):
        self.im_in_plot = 0
        self.data = []
        
    def disp_img_1d(self,img,hold=False, save=False, nr=0 , ret=False):
        self.data.append(img)
        p = '/home/jonfrey/Debug/Midterm2/'
        
        if not hold:
            fig = plt.figure(figsize=(6*2*len(self.data),7))
            ax = []
            for j,a in enumerate(self.data):
                ax.append( fig.add_subplot(1,len(self.data), j+1)  )
                
                ax[-1].get_xaxis().set_visible(False)
                ax[-1].get_yaxis().set_visible(False)
                pos = ax[-1].imshow( a, cmap='Reds' )
                try:
                    if a.shape[2] != 3:
                        fig.colorbar(pos, ax=ax[-1])
                except:
                    pass
            plt.show()
            if save:
                fig.savefig(p+str(nr)+'.png', dpi=300)
                
            if ret: 
                if isinstance( self.data[0], torch.Tensor):
                    self.data[0] = self.data[0].numpy()
                    print('CONV')
                    
                print(self.data[0].shape)
                a = np.max(self.data[0])
                b = np.min(self.data[0])
                
                d = (self.data[0]-float(b))
                d = (d / ((float(a)-float(b))) )*255 
                d = np.uint8(d)
                img = Image.fromarray( d )
                return d
            self.data = []
            self.ax = []
            
Nc = 256
cmap = plt.cm.get_cmap('gist_rainbow', Nc)
cmaplist = [cmap(i) for i in range(cmap.N)]

def disp_alignment(depth, label, real):
    data = np.zeros((480,640,4), dtype=np.uint8)
    data_depth = np.zeros((480,640,4), dtype=np.uint8)
    t = real
    data[:,:,:3] = t.numpy() # red patch in upper left
    data_depth[:,:,:3] = t.numpy()
    data[:,:,3] = 70
    data[:,:,3][label==8] = 255
    
    min_val = torch.min( depth[depth!=0] )
    max_val = torch.max( depth[depth!=0] )
    val = torch.clamp( ((depth-min_val) // (max_val-min_val))*255, 0, 255)
    
    img = Image.fromarray(data, 'RGBA')
    display(img)

def plot_mask(mask):
    min_val = torch.min( mask )
    max_val = float( max(1,torch.max( mask )) )
    mask = torch.clamp( (mask-min_val) / (max_val-min_val)*255 ,0,255)
    
    data_depth = np.zeros((480,640,4), dtype=np.uint8)
    data_depth[:,:,3] = 255
    for i in range(480):
        for j in range(640):
            data_depth[i,j,:4] = np.array( cmaplist[ int(mask[i,j])] )*255
    data_depth[:,:,3] = 255
    data_depth[:,:,3][label==2] = 255
    img_depth = Image.fromarray(data_depth, 'RGBA')
    display(img_depth)

def plot_two_pcd_line(x, y, point_size=0.005, c1='g', c2='r'):
    if c1 == 'b':
        k = 245
    elif c1 == 'g':
        k = 25811000
    elif c1 == 'r':
        k = 11801000
    elif c1 == 'black':
        k = 2580
    else:
        k = 2580

    if c2 == 'b':
        k2 = 245
    elif c2 == 'g':
        k2 = 25811000
    elif c2 == 'r':
        k2 = 11801000
    elif c2 == 'black':
        k2 = 2580
    else:
        k2 = 2580

    col1 = np.ones(x.shape[0]) * k
    col2 = np.ones(y.shape[0]) * k2
    plot = k3d.plot(name='points')
    plt_points = k3d.points(x, col1.astype(np.uint32), point_size=point_size)
    plot += plt_points
    plt_points = k3d.points(y, col2.astype(np.uint32), point_size=point_size)
    plot += plt_points
    for i in range(min(100,x.shape[0]) ):
        plot += k3d.line([x[i],y[i]],shader='mesh', width=0.0005, color=0xff0000)
    
    plt_points.shader = '3d'
    plot.display()

def plot_hist(x,n_bins = 20):
    fig, axs = plt.subplots(1, 1, sharey=True, tight_layout=True)
    colors = ['lime']
    axs.hist(x, bins=n_bins, color=colors, label=colors)
    fig.show()
    return fig

In [ ]:
# Helper functions 
def get_scale_for_erosion(ero_in):
    res = torch.sum ( ero_in, dim = (2,3))
    res[res < 5000] = 5
    res[res < 10000] = 10
    res[res < 30000] = 20
    res[res < 40000] = 25
    res[res < 50000] = 30
    res[res >= 50000] = 40
    return res

def eroision(t,size=3):
    "t: tensor shape BS, C, H,W"
    out_c = t.shape[1]
    kernel_tensor = torch.ones( (out_c,1,size,size) )
    print(size, kernel_tensor, t.shape)
    return torch.nn.functional.conv2d(t, kernel_tensor, padding=(int((size)/2), int((size)/2))) == (size*size)

def eroision_batch(t,t_size):
    "t: tensor shape BS, C, H,W"
    "t_size: tensor shape BS"
    out_c = t.shape[1]
    for b in range( t.shape[0] ):
        size = int( t_size[b] )
        kernel_tensor = torch.ones( (out_c,1,size,size) )
        t[b] = (torch.nn.functional.conv2d(t[b][None], kernel_tensor, padding=(int((size)/2), int((size)/2))) == (size*size))[0,:,:t.shape[2], :t.shape[3]]
    return t



In [ ]:
# How to load a specific datapoint form the dataset (Not Needed)

# [('data/0003/001742',), tensor([8], dtype=torch.int32)]

desig = unique_desig[0]
desig = 'data/0003/001742'

_p_ycb = "/media/scratch1/jonfrey/datasets/YCB_Video_Dataset"
depth = np.array(Image.open(
    '{0}/{1}-depth.png'.format(_p_ycb, desig)))
depth.shape

label = np.array(Image.open(
    '{0}/{1}-label.png'.format(_p_ycb, desig)))
img = np.array(Image.open(
    '{0}/{1}-color.png'.format(_p_ycb, desig)))
batch = dataset_train._backend.getElement( desig, 8)
batch = batch #bann 10450   
model_points = batch[4]
idx = batch[5]  # Be carefull here the first objects starts with 0. Normally 0 is the NO object class in all other datastructures
real_img_original = batch[8]
cam = batch[9]
real_img, render_img, real_d, render_d, gt_label_cropped = batch[13:18]
pred_rot_wxyz, pred_trans, pred_points, h_render, h_real, render_img_original = batch[18:24]
u_map, v_map, flow_mask, bb, render_orig = batch[24:]


In [ ]:
Nc = 256
cmap = plt.cm.get_cmap('gist_rainbow', Nc)
cmaplist = [cmap(i) for i in range(cmap.N)]
import time

In [ ]:
# set loader to BS 1
exp['loader']['batch_size'] = 1
exp['loader']['pin_memory'] = False
exp['loader']['shuffle'] = True
exp['loader']['num_workers'] = 1

exp['d_train']["output_cfg"]['overfitting_nr_idx'] = -1
exp['d_train']["flow_cfg"]['sub'] = 1
exp['d_train']["flow_cfg"]['min_matches'] = 50
exp['d_train']["flow_cfg"]['max_matches'] = 5000
exp['d_train']["flow_cfg"]['max_iterations'] = 20000
exp['d_train']["flow_cfg"]['dil_kernel_size'] = 3

dataset_train = GenericDataset(
    cfg_d=exp['d_train'],
    cfg_env=env)


# get test and train dataset
dataloader_train = torch.utils.data.DataLoader(dataset_train,
                                                       **exp['loader'])
exp['d_test']["output_cfg"]['overfitting_nr_idx'] = -1
exp['d_test']["output_cfg"]['noise_translation'] = 0.02
exp['d_test']["output_cfg"]['noise_rotation'] = 40

exp['d_test']["flow_cfg"]['sub'] = 1
exp['d_test']["flow_cfg"]['min_matches'] = 300
exp['d_test']["flow_cfg"]['max_matches'] = 5000
exp['d_test']["flow_cfg"]['max_iterations'] = 20000
exp['d_test']["flow_cfg"]['dil_kernel_size'] = 1
dataset_test = GenericDataset(
    cfg_d=exp['d_test'],
    cfg_env=env)
dataloader_test = torch.utils.data.DataLoader(dataset_test,
                                                       **exp['loader'])

# get Loss function
criterion_adds = LossAddS(sym_list=exp['d_train']['obj_list_sym'])





In [ ]:
# Numpy ICP implementation from https://github.com/ClayFlannigan/icp/blob/master/icp.py

import numpy as np
from sklearn.neighbors import NearestNeighbors


def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform that maps corresponding points A to B in m spatial dimensions
    Input:
      A: Nxm numpy array of corresponding points
      B: Nxm numpy array of corresponding points
    Returns:
      T: (m+1)x(m+1) homogeneous transformation matrix that maps A on to B
      R: mxm rotation matrix
      t: mx1 translation vector
    '''

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
       Vt[m-1,:] *= -1
       R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(m+1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t


def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nxm array of points
        dst: Nxm array of points
    Output:
        distances: Euclidean distances of the nearest neighbor
        indices: dst indices of the nearest neighbor
    '''

    assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()


def icp(A, B, init_pose=None, max_iterations=20, tolerance=0.001):
    '''
    The Iterative Closest Point method: finds best-fit transform that maps points A on to points B
    Input:
        A: Nxm numpy array of source mD points
        B: Nxm numpy array of destination mD point
        init_pose: (m+1)x(m+1) homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation that maps A on to B
        distances: Euclidean distances (errors) of the nearest neighbor
        i: number of iterations to converge
    '''

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1,A.shape[0]))
    dst = np.ones((m+1,B.shape[0]))
    src[:m,:] = np.copy(A.T)
    dst[:m,:] = np.copy(B.T)

    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        # find the nearest neighbors between the current source and destination points
        distances, indices = nearest_neighbor(src[:m,:].T, dst[:m,:].T)

        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[:m,:].T, dst[:m,indices].T)

        # update the current source
        src = np.dot(T, src)

        # check error
        mean_error = np.mean(distances)
        if np.abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error

    # calculate final transformation
    T,_,_ = best_fit_transform(A, src[:m,:].T)

    return T, distances, i

In [ ]:
idxmax = -1 
def get_H(pcd):
    pcd_ret = torch.ones( (pcd.shape[0],pcd.shape[1]+1),device=pcd.device, dtype=pcd.dtype )
    pcd_ret[:,:3] = pcd
    return pcd_ret

def eval_T(P_real_in_center, P_ren_in_center, T_res):
        """
        NR,3
        NR,3 
        4,4
        """
        P_ren_H = get_H( P_ren_in_center )
        P_ren_trafo =  (P_ren_H @ T_res.T)[:,:3]
        L2_dis_post = torch.mean( torch.norm( P_real_in_center-P_ren_trafo, dim=1 ) )
        L2_dis_pre = torch.mean( torch.norm( P_real_in_center-P_ren_in_center, dim=1 ) )
        return L2_dis_post, L2_dis_pre  

In [ ]:
import torch
import numpy as np
import copy
from helper import anal_tensor

def solve_transform(keypoints, gt_keypoints):
    """
    keypoints: N x K x 3
    gt_keypoints: K x 3
    return: N x 4 x 4 transformation matrix
    """
    try:
        keypoints = keypoints.clone()
        gt_keypoints = gt_keypoints.clone()
        N, K, _ = keypoints.shape
        center = keypoints.mean(dim=1)
        gt_center = gt_keypoints.mean(dim=0)
        keypoints -= center[:, None, :]
        gt_keypoints -= gt_center[None]
        matrix = keypoints.transpose(2, 1) @ gt_keypoints[None]
        U, S, V = torch.svd(matrix)
        
        Vt = V.transpose(2, 1)
        Ut = U.transpose(2, 1)

        d = (V @ Ut).det()
        I = torch.eye(3, 3, dtype=gt_center.dtype, device= keypoints.device)[None].repeat(N, 1, 1)
        I[:, 2, 2] = d.clone()

        R = U @ I @ Vt
        T = torch.zeros(N, 4, 4, dtype=gt_center.dtype, device= keypoints.device)
        T[:, 0:3, 0:3] = R
        T[:, 0:3, 3] = center[None] - (R @ gt_center[None :, None])[:, :, 0]
        T[:, 3, 3] = 1.0

        return T
    except RuntimeError as error:
        import ipdb; ipdb.set_trace()
        print("Something went wrong")

# costume implementation 
def solve_transform2(A,B):
    if A.shape[0] > B.shape[0]:
        x=torch.arange(A.shape[0],device=A.device)
        out = torch.randperm(x.numel(),device=A.device)[:B.shape[0]]
        A = torch.index_select(A, 0, out)
    if A.shape[0] < B.shape[0]:
        x=torch.arange(B.shape[0],device=A.device)
        out = torch.randperm(x.numel(),device=A.device)[:A.shape[0]]
        B = torch.index_select(B, 0, out)

    #A = torch.choice(A,B.shape[0])

    assert A.shape == B.shape

    m = A.shape[1]
    centroid_A = torch.mean(A, dim=0)
    centroid_B = torch.mean(B, dim=0)

    AA = (A - centroid_A)
    BB = (B - centroid_B)
    H = AA.transpose(0,1) @ BB
    U, S, Vt = torch.svd(H)
    R = Vt @ U.transpose(0,1)
    if torch.det(R) < 0:
        Vt[m-1,:] *= -1
        R = Vt.transpose(0,1) @ U.transpose(0,1)

    # translation
    t = centroid_B - (R @ centroid_A)
    # homogeneous transformation
    T = torch.eye(m+1, device= A.device)
    T[:m, :m] = R
    T[:m, m] = t
    return T


# NR = 1000
# DIM = 3
# A = torch.ones( (NR,DIM), dtype= torch.float32)
# B = A*1.7223
# T = solve_transform2(A,B)
# print(T)


# A_hom = get_H(A)
# A_hom2 = A_hom @ T.T
# print(A_hom2[:,:3])

def filter_pcd_given_depthmap(pcd, depth, scal= 10000):
    """
    pcd = Nx3 troch.float32
    depth = N torch.float32

    return N torch.bool
    """
    m1 = (depth/scal) > 0.01
    #print( "Thorwn away values", (depth/scal) < 0.2 )
    return m1

    val_d = depth[ m1 ]
    mean = torch.mean(val_d)
    new_d = depth - mean
    tol = 0.5
    m2 = torch.abs( new_d/scal ) < tol 
    return m1 * m2
    
def filter_pcd( pcd, tol = 0.6):
    """
    input:
        pcd : Nx3 torch.float32
    returns:
        mask : NX3 torch.bool 
    """
    return pcd[:,2] > 0.05
    
    m = torch.mean(pcd, dim = 0)
    comp = m[None,:].repeat(pcd.shape[0],1) + tol
    mean_free = pcd-m[None,:].repeat(comp.shape[0],1)
    mask = torch.norm( mean_free,  dim= 1) > tol
    #print(f"filter_pcd PRE: {pcd.shape}, POST: {float(torch.sum(mask[:,None].repeat(1,3) == False ))/3.0}")
    return mask == False

def filter_pcd_cor(pcd1, pcd2, max_mean_deviation=0.2):
    
    dif = torch.norm( pcd1-pcd2 , dim= 1)
    mean = torch.mean(dif, dim = 0)
    mean_free = torch.abs(dif-mean)
    #print(f"filter_pcd_cor PRE: {pcd1.shape[0]}, POST: {torch.sum(mean_free < max_mean_deviation)}")
    return mean_free < max_mean_deviation
    
def flow_to_trafo(*args, **kwargs):
    """
    input:
      real_br: torch.tensor torch.Size([2])
      real_tl: torch.tensor torch.Size([2])
      ren_br: torch.tensor torch.Size([2])
      ren_tl: torch.tensor torch.Size([2])
      flow_mask: torch.Size([480, 640])
      u_map: torch.Size([480, 640])
      v_map: torch.Size([480, 640])
      K_real: torch.Size([3, 3])
      K_ren: torch.Size([3, 3])
      real_d: torch.Size([480, 640]) 
      render_d: torch.Size([480, 640])
      h_real: torch.Size([4, 4])
      h_render: torch.Size([4, 4])
    output:
      P_real_in_center: torch.Size([N, 3])
      P_ren_in_center: torch.Size([N, 3]) 
      P_real_trafo: torch.Size([N, 3])
      T_res: torch.Size([4, 4])
      
      The output rotation T_res is defined in the Camera coordinate frame. 
      Therfore premultiply the T_Res with h_render to get the new h_real_new !!!
    """
    for k in kwargs.keys():
        pass
        #print(f"Variable: {k}, Type {type(kwargs[k])}, Dtype{kwargs[k].dtype}, Shape{kwargs[k].shape}")
    real_br = kwargs['real_br']
    real_tl = kwargs['real_tl']
    ren_br = kwargs['ren_br']
    ren_tl = kwargs['ren_tl']
    flow_mask = kwargs['flow_mask']
    u_map = kwargs['u_map']
    v_map = kwargs['v_map']
    K_real = kwargs['K_real']
    K_ren = kwargs['K_ren']
    real_d = kwargs['real_d']
    render_d = kwargs['render_d']
    h_real = kwargs['h_real']
    h_render = kwargs['h_render']
    plot_pcd = kwargs.get('plot_pcd',False)
  
    # Grid for upsampled real
    grid_real_h = torch.linspace(int(real_tl[0]) ,int(real_br[0]) , 480, device=u_map.device)[:,None].repeat(1,640)
    grid_real_w = torch.linspace(int(real_tl[1]) ,int(real_br[1]) , 640, device=u_map.device)[None,:].repeat(480,1)


    # Grid for upsampled ren
    c = 0
    
    grid_ren_h = torch.linspace(int(ren_tl[0]) ,int(ren_br[0]) , 480, device=u_map.device)[:,None].repeat(1,640)
    grid_ren_w = torch.linspace(int(ren_tl[1]) ,int(ren_br[1]) , 640, device=u_map.device)[None,:].repeat(480,1)
    # Calculate valid depth map for rendered image
    render_d_ind_h = torch.linspace(0 ,479 , 480, device=u_map.device)[:,None].repeat(1,640)
    render_d_ind_w= torch.linspace(0 ,639 , 640, device=u_map.device)[None,:].repeat(480,1)

    render_d_ind_h = torch.clamp(torch.round((render_d_ind_h - u_map).type(torch.float32)) ,0,479).type( torch.long )[flow_mask]
    render_d_ind_w = torch.clamp(torch.round((render_d_ind_w - v_map).type(torch.float32)),0,639).type( torch.long )[flow_mask] 
    index = render_d_ind_h*640 + render_d_ind_w # hacky indexing along two dimensions
    ren_d_masked  = render_d.flatten()[index]
    
    # Project depth map to the pointcloud real
    cam_scale = 10000

    real_pixels = torch.stack( [grid_real_w[flow_mask], grid_real_h[flow_mask], torch.ones(grid_real_h.shape, device = u_map.device,  dtype= u_map.dtype)[flow_mask]], dim=1 ).type(u_map.dtype)
    K_inv = torch.inverse(K_real.type(torch.float32)).type(u_map.dtype)
    P_real = K_inv @ real_pixels.T
    P_real = P_real * real_d[flow_mask] / cam_scale
    P_real = P_real.T
    
    # Project depth map to the pointcloud render
    K_ren_inv = torch.inverse(K_ren.type(torch.float32)).type(u_map.dtype)
    ren_pixels = torch.stack( [grid_ren_w[flow_mask] - v_map[flow_mask], 
                            grid_ren_h[flow_mask] - u_map[flow_mask],
                            torch.ones(grid_ren_h.shape, device = u_map.device,  dtype= u_map.dtype )[flow_mask]], 
                            dim=1 ).type(u_map.dtype)
    P_ren = K_ren_inv @ ren_pixels.T
    P_ren = P_ren * ren_d_masked / cam_scale
    P_ren = P_ren.T

    # Filter the pointclouds given the depthmap
#     m_ren_depth = filter_pcd_given_depthmap(P_ren, ren_d_masked)
#     m_real_depth = filter_pcd_given_depthmap(P_real, real_d[flow_mask])
#     m_total =  m_ren_depth * m_real_depth
    
    min_points = 20
#     if torch.sum(m_total) < min_points:
#         print(f'Violation filter pcd_given_depthmap: P_in: {P_ren.shape[0]} P_out: {torch.sum(m_total)}')
#         return False, P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)

#     P_ren = P_ren[m_total] 
#     P_real = P_real[m_total]
    # anal_tensor(  P_ren, 'P_ren m_total masked')

    # Do not transfrom to center coordinate system
    P_real_in_center = P_real                      
    P_ren_in_center = P_ren 
    
    m_real = filter_pcd( P_real_in_center )
    m_ren = filter_pcd( P_ren_in_center )
    m_tot = m_real * m_ren
    if torch.sum(m_tot) < min_points:
        print(f'Violation filter_pcd: P_in: { P_ren_in_center.shape[0]} P_out: {torch.sum(m_tot)}')
        return False, P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)

    P_real_in_center = P_real_in_center[m_tot]
    P_ren_in_center = P_ren_in_center[m_tot]
  
    # Max mean deviation
    m_new = filter_pcd_cor(P_real_in_center, P_ren_in_center)
    
    if torch.sum(m_new) < min_points:
        print(f'Violation filter_pcd_cor: P_in: { P_ren_in_center.shape[0]} P_out: {torch.sum(m_new)}')
        return False, P_real, P_ren, P_real, torch.eye(4, dtype= u_map.dtype, device=u_map.device)

    P_real_in_center = P_real_in_center[m_new]
    P_ren_in_center = P_ren_in_center[m_new]

    # random shuffel
    pts_trafo = min( P_real_in_center.shape[0], 1000 )
    idx = torch.randperm( P_real_in_center.shape[0] )[0:pts_trafo]
    P_real_in_center = P_real_in_center[idx]
    P_ren_in_center = P_ren_in_center[idx]

    T_res = solve_transform( P_real_in_center[None].type(torch.float64 ) , P_ren_in_center.type(torch.float64 ) ).type(u_map.dtype )
    
    # Transform the real points according to calculated transformation
    P_hr = torch.ones( (P_real_in_center.shape[0],4 ) , device=u_map.device, dtype= u_map.dtype)
    P_hr[:,:3] = P_real_in_center
    P_real_trafo = (torch.inverse( T_res[0].type(torch.float32) ).type(u_map.dtype ) @ copy.deepcopy(P_hr).T).T [:,:3]

    return True, P_real_in_center, P_ren_in_center, P_real_trafo, T_res[0]



In [ ]:
import cv2 
cv2.__version__
from scipy.spatial.transform import Rotation as R
import k3d 
def rvec_tvec_to_H(r_vec,t_vec):
    # get homogenous output transformation
    rot = R.from_rotvec(r_vec)
    h = np.eye(4)
    h[:3,:3] = rot.as_matrix()
    h[:3,3] = t_vec.T
#     print(h)
    return h
class SingleObjectADDLoss:
    def asymmetric(self, gt_T, T_hat, model_points):
        R_hat = T_hat[:3, :3]
        t_hat = T_hat[:3, 3, None]
        model_points = model_points[:, :, None]
        predicted = ((R_hat @ model_points) + t_hat)[:, :, 0]
        R_gt = gt_T[:3, :3]
        t_gt = gt_T[:3, 3, None]
        ground_truth = ((R_gt @ model_points) + t_gt)[:, :, 0]
        return (ground_truth - predicted).norm(dim=1).mean()

    def symmetric(self, gt_T, T_hat, model_points):
        ones = torch.ones(
            model_points.shape[0], 1, dtype=model_points.dtype).to(gt_T.device)
        points = torch.cat([model_points, ones], dim=1)[:, :, None]
        ground_truth = (gt_T @ points)[:, :3, 0]
        predicted = (T_hat @ points)[:, :3, 0]
        dima = (ground_truth[None] - predicted[:, None]).norm(dim=2)
        min_values, _ = dima.min(dim=1)
        return min_values.mean(dim=0)
    
    def pcd(self, ground_truth, predicted):
        dima = (ground_truth[None] - predicted[:, None]).norm(dim=2)
        min_values, _ = dima.min(dim=1)
        return min_values.mean(dim=0)
    
def flow_to_trafo_pnp(*args, **kwargs):
    """
    input:
      real_br: torch.tensor torch.Size([2])
      real_tl: torch.tensor torch.Size([2])
      ren_br: torch.tensor torch.Size([2])
      ren_tl: torch.tensor torch.Size([2])
      flow_mask: torch.Size([480, 640])
      u_map: torch.Size([480, 640])
      v_map: torch.Size([480, 640])
      K_real: torch.Size([3, 3])
      K_ren: torch.Size([3, 3])
      real_d: torch.Size([480, 640]) 
      render_d: torch.Size([480, 640])
      h_real: torch.Size([4, 4])
      h_render: torch.Size([4, 4])
    output:
      P_real_in_center: torch.Size([N, 3])
      P_ren_in_center: torch.Size([N, 3]) 
      P_real_trafo: torch.Size([N, 3])
      T_res: torch.Size([4, 4])
      
      The output rotation T_res is defined in the Camera coordinate frame. 
      Therfore premultiply the T_Res with h_render to get the new h_real_new !!!
    """
    real_br = kwargs['real_br']
    real_tl = kwargs['real_tl']
    ren_br = kwargs['ren_br']
    ren_tl = kwargs['ren_tl']
    flow_mask = kwargs['flow_mask']
    u_map = kwargs['u_map']
    v_map = kwargs['v_map']
    K_real = kwargs['K_real']
    K_ren = kwargs['K_ren']
    real_d = kwargs['real_d']
    render_d = kwargs['render_d']
    h_real = kwargs['h_real']
    h_render = kwargs['h_render']
    h_real_est = kwargs['h_real_est']
    mp = kwargs['mp']
    img = kwargs['img']

    # Grid for upsampled real
    grid_real_h = torch.linspace(int(real_tl[0]) ,int(real_br[0]) , 480, device=u_map.device)[:,None].repeat(1,640)
    grid_real_w = torch.linspace(int(real_tl[1]) ,int(real_br[1]) , 640, device=u_map.device)[None,:].repeat(480,1)

    # Grid for upsampled ren
    c = 0
    grid_ren_h = torch.linspace(int(ren_tl[0]) ,int(ren_br[0]), 480, device=u_map.device)[:,None].repeat(1,640)
    grid_ren_w = torch.linspace(int(ren_tl[1]) ,int(ren_br[1]) , 640, device=u_map.device)[None,:].repeat(480,1)
    # Calculate valid depth map for rendered image
    render_d_ind_h = torch.linspace(0 ,479 , 480, device=u_map.device)[:,None].repeat(1,640)
    render_d_ind_w= torch.linspace(0 ,639 , 640, device=u_map.device)[None,:].repeat(480,1)


    render_d_ind_h = torch.clamp((render_d_ind_h - u_map).type(torch.float64) ,0,479).type( torch.long )[flow_mask]
    render_d_ind_w = torch.clamp((render_d_ind_w - v_map).type(torch.float32),0,639).type( torch.long )[flow_mask] 
    index = render_d_ind_h*640 + render_d_ind_w # hacky indexing along two dimensions

    ren_d_masked  = render_d.flatten()[index]

    # Project depth map to the pointcloud real
    cam_scale = 10000

    real_pixels = torch.stack( [grid_real_w[flow_mask], grid_real_h[flow_mask], torch.ones(grid_real_h.shape, device = u_map.device,  dtype= u_map.dtype)[flow_mask]], dim=1 ).type(u_map.dtype)
    K_inv = torch.inverse(K_real.type(torch.float64))
    P_real = K_inv @ real_pixels.T.type(torch.float64)
    P_real = P_real.type(torch.float64) * real_d[flow_mask] / cam_scale
    P_real = P_real.T

    # Project depth map to the pointcloud render
    K_ren_inv = torch.inverse(K_ren.type(torch.float64)).type(torch.float64)
    ren_pixels = torch.stack( [grid_ren_w[flow_mask] - v_map[flow_mask], 
                            grid_ren_h[flow_mask] - u_map[flow_mask],
                            torch.ones(grid_ren_h.shape, device = u_map.device,  dtype=torch.float64 )[flow_mask]], 
                            dim=1 ).type(torch.float64)

    print(K_ren_inv, K_inv)
    cam_scale = 10000
    P_ren = K_ren_inv @ ren_pixels.T
    P_ren = P_ren * ren_d_masked / cam_scale
    P_ren = P_ren.T


    # m_ren = filter_pcd( P_ren)
    # min_points = 50
    # if torch.sum(m_ren) < min_points:
    #     print(f'Violation filter_pcd: P_in: { P_ren_in_center.shape[0]} P_out: {torch.sum(m_tot)}')


    # P_real = P_real[m_ren]
    # P_ren = P_ren[m_ren]
    real_pixels_fil = real_pixels #[m_ren]


    grid_ren_h = torch.linspace(int(ren_tl[0]) ,int(ren_br[0]), 480, device=u_map.device)[:,None].repeat(1,640)
    grid_ren_w = torch.linspace(int(ren_tl[1]) ,int(ren_br[1]) , 640, device=u_map.device)[None,:].repeat(480,1)
    crop_d_pixels = torch.stack( [grid_ren_w.flatten(), grid_ren_h.flatten(), torch.ones(grid_ren_w.shape, device = u_map.device,  dtype= u_map.dtype).flatten()], dim=1 ).type(u_map.dtype)
    K_inv = torch.inverse(K_ren.type(torch.float64)).type(u_map.dtype)

    P_crop_d = K_inv @ crop_d_pixels.T.type(u_map.dtype)
    P_crop_d = P_crop_d.type(u_map.dtype) * render_d.flatten() / cam_scale
    P_crop_d = P_crop_d.T
    P_crop_d  = P_crop_d[index] 
    m = filter_pcd( P_crop_d)
    P_crop_d  = P_crop_d[ m ]
    P_real = P_real[m]

    P_ren = P_crop_d
    print(P_ren.shape, P_real.shape,index.shape)

    # random shuffel

    pts_trafo = min(P_real.shape[0], 50000)
    idx = torch.randperm( P_real.shape[0] )[0:pts_trafo]
    P_real = P_real[idx]
    P_ren = P_ren[idx]
    real_pixels_fil = real_pixels_fil[idx]
    
#     #################################################################
#     ################# Evaluate Depth Map ############################
     
#     adds = SingleObjectADDLoss()
    
#     P_ren_h2 = get_H(P_ren.clone())
#     P_ren_h_trafo2 = P_ren_h2 @ torch.inverse(h_render.clone()).T
#     a = torch.abs(P_ren_h_trafo2[:,:3])
#     m1 = a < 0.1
#     m1 = torch.sum(m1.type(torch.float32), dim=1 ) > 2
#     P_ren_h_trafo2 = P_ren_h_trafo2[m1]
    
    
#     #ken_adds = adds.pcd(P_ren_h_trafo2[:,:3], mp)
    
#     pts = min( P_ren_h_trafo2.shape[0], 1000)
#     idx_de = torch.randperm(  P_ren_h_trafo2.shape[0] )[0:pts]
#     idx_mp = torch.randperm( pts )[0:pts]
#     id_sym = exp['d_train']['obj_list_sym'][0]
#     ids = torch.tensor([[id_sym]])
#     st = time.time()
#     adds_error = criterion_adds(P_ren_h_trafo2[idx_de][:,:3][None].clone(), mp[idx_mp][None].clone(), idx=ids, H = torch.eye(4)[None])
#     ids = torch.tensor([[999]])
#     st2 = time.time()
    
#     print(F'adds_error {adds_error} s1 {time.time()-st}, s2 {time.time()-st2}')
    
#     if adds_error > 0.02:
#         plot_two_pcd_line(P_ren_h_trafo2[idx_de][:,:3].numpy(), mp[idx_mp].clone().numpy() )
    
#     ##################################################################

    # random shuffel
 
    
    sub = int(P_ren.shape[0]/200)
    tar = get_H(mp)
    tar = (tar @ h_render.T)[::5,:3]
    print("Check alignment betweem rendered pointcloud and the cad model rotated to the pose of the PCD")
    plot_two_pcd(P_ren[::sub,:3].numpy(), tar.numpy())
    
    # The same filtering is performed for the PnP as for the Pointcloud matching approach! 
    
    
    # Transform the known rendered point corrspondences to the origin frame. 
    P_ren_h = get_H(P_ren)
    P_ren_h_trafo = P_ren_h.clone() @ torch.inverse(h_render).T
    
    
    mp_real = get_H(mp)
    mp_real = (mp_real @ h_real.T)[::5,:3]

    # PNP estimation
    objectPoints = P_ren_h_trafo[:,:3].clone().numpy() #P_ren.numpy()    
    imagePoints = real_pixels_fil[:,:2].numpy()
    dist = np.array( [[0.0,0.0,0.0,0.0]] )
    div = 1 # int(objectPoints.shape[0]/1000)

    if objectPoints.shape[0] < 8:
        print('Failed due to missing corsspondences')
    # set current guess as the inital estimate

    rvec = R.from_matrix(h_real_est[:3,:3]).as_rotvec().astype(np.float32)
    tvec = h_real_est[:3,3].numpy().astype(np.float32)
    # calculate PnP between the pixels coordinates in the real image and the corrosponding points in the origin frame
    r_vec2, t_vec2 = cv2.solvePnPRefineLM(copy.deepcopy(objectPoints), \
        copy.deepcopy(imagePoints), 
        K_real.numpy(), 
        dist, 
        copy.deepcopy(rvec),
        copy.deepcopy( tvec))
    r_vec2 = r_vec2[:,None]
    h = rvec_tvec_to_H(r_vec2[:,0],t_vec2)
    print("H predicted \n",h, "\n H real \n", h_real)
    P_ren_h_trafo_out = P_ren_h_trafo @ h.T
    sub = int(P_ren_h_trafo_out.shape[0]/500 )
    plot_two_pcd_line(P_ren_h_trafo_out[::sub,:3].numpy(), mp_real.clone().numpy() )
    
#     # plotting
#     img = np.uint8(img.clone().numpy())
#     for p in range(imagePoints.shape[0]):
        
#         u = min(479,int(imagePoints[p,1]) )
#         v = min(639,int(imagePoints[p,0]) )
#         img[u,v, :] = np.array([138,43,226])
    
#     Drawer().disp_img_1d(img, hold=False )
#     visualizer.plot_estimated_pose( tag = f"_",
#         epoch = 1,
#         img= img,
#         points = P_ren_h_trafo_out[:2,:3],
#         store = False,
#         jupyter= True,
#         K = K_real.cpu().numpy(),
#         H = np.eye(4),
#         method='right')
    
#     visualizer.plot_estimated_pose( tag = f"_",
#         epoch = 1,
#         img= img,
#         points = P_ren_h_trafo_out[:,:3],
#         store = False,
#         jupyter= True,
#         K = K_real.cpu().numpy(),
#         H = np.eye(4),
#         method='left')

    
    
    return True, torch.from_numpy( h).type(u_map.dtype) 
# print(model_points[0].shape, real_img.shape)

flow_to_trafo_pnp(
    real_br = real_br[b].clone(),
    real_tl = (real_tl[b]).clone(), 
    ren_br = (ren_br[b]).clone(), 
    ren_tl = (ren_tl[b]).clone(),
    flow_mask = (flow_mask[b]).clone(), 
    u_map = (u_map[b].type( typ )).clone(),
    v_map = (v_map[b].type( typ )).clone(), 
    K_real = (K_real.type( typ )).clone(),
    K_ren = (K_ren.type( typ )).clone(),
    real_d = (real_d[b].type( typ )).clone(),
    render_d = (render_d[b].type( typ )).clone(),
    h_real = (h_real[b].type( typ )).clone(), 
    h_render = (h_render[b].type( typ )).clone(),
    h_real_est = h_real_est,
    mp = model_points[0],
    img = real_img_original[0].clone())
# print("DONE")

In [ ]:
from visu import plot_two_pcd

import pandas as pd
# idxmax can be set to != -1 to load specific sample

idxmax = -1 
device = 'cpu'
desig_ls= []
adds_gt_ls = []
adds_init_ls = []
adds_erode_ls = []
adds_n1_ls = []
adds_n2_ls = []

adds_pnp_gt_ls = []
adds_pnp_ero_ls = []

failed_desig_ls = []
visualizer = Visualizer('/home/jonfrey/Debug', None)
print('START')

K_ren = torch.tensor( dataset_train._backend.get_camera('data_syn/0019', K=True), device=device ) 
K_ren = torch.tensor( dataset_train._backend.get_camera('data/0049', K=True), device=device ) 

max_iter = 1000

if idxmax != -1:
    print(idxmax, "is not equal to -1 load batch")
    batch = next(itertools.islice(dataloader_train, idxmax, None))[0]
cou = 0
print("START")
names = ['ID','ADD-S GT','ADD-S INITAL','ADD-S Eroded Mask', 'ADD-S Random', 'ADD-S Bias', 'ADD-S PNP GT', 'ADD-S PNP Ero']
df = pd.DataFrame(columns=names)
        
for j,ba in enumerate(dataloader_test):
    
    if j > max_iter-1:
        break
    if j % 10 == 0:
        print(f"Processed {j}/{max_iter}")

    if idxmax == -1:  
        batch = ba[0]
    # use first sample in batch 
    b = 0
    
    model_points = batch[4]
    idx = batch[5]  # Be carefull here the first objects starts with 0. Normally 0 is the NO object class in all other datastructures
    label = batch[7]
    real_img_original = batch[8]
    cam = batch[9]
    gt_rot_wxyz, gt_trans, unique_desig = batch[10:13] # unique_desig[1] contains the idx starting at 1 for the first object 
    bs = model_points.shape[0]
    if batch[13] is False:
        print('Continue')
        continue
    real_img, render_img, real_d, render_d, gt_label_cropped = batch[13:18]
    pred_rot_wxyz, pred_trans, pred_points, h_render, h_real, render_img_original = batch[18:24]
    u_map, v_map, flow_mask, bb, depth_render_original= batch[24:]
    real_tl, real_br, ren_tl, ren_br = bb 
    
    data = torch.cat([real_img, render_img], dim=1)
    uv_gt = torch.stack( [u_map, v_map], dim=3 ).permute(0,3,1,2)
    
    
    ero_in = (gt_label_cropped ==  unique_desig[1])[:,None,:,:].type(torch.float32) # BS,C,H,W
    t_size = get_scale_for_erosion(ero_in)
    ero_out = eroision_batch(ero_in,t_size)
    
    # get camera
    K_real = torch.tensor( [[cam[b,2],0,cam[b,0]],[b,cam[b,3],cam[b,1]],[0,0,1]], device=device )
    
    #get inital estimate of the poistion given by the dataloader
    h_real_est = torch.eye(4,device=device)
    h_real_est[:3,:3] = quat_to_rot(pred_rot_wxyz[b][None,:], conv='wxyz', device=device)
    h_real_est[:3,3] = torch.tensor( pred_trans[b].clone().detach() ,device=device )
    
    #print(h_render)
    #break
    
    ### GT Estimate ###
    typ = u_map.dtype
    fmt = flow_mask.dtype
    flow_mask_eroded  = (flow_mask * ero_out.type(torch.float32)).type(fmt)[:,0]
    suc = True
    suc_, pnp_h_gt = pnp(
        real_br = real_br[b].clone(),
        real_tl = (real_tl[b]).clone(), 
        ren_br = (ren_br[b]).clone(), 
        ren_tl = (ren_tl[b]).clone(),
        flow_mask = (flow_mask[b]).clone(), 
        u_map = (u_map[b].type( typ )).clone(),
        v_map = (v_map[b].type( typ )).clone(), 
        K_real = (K_real.type( typ )).clone(),
        K_ren = (K_ren.type( typ )).clone(),
        real_d = (real_d[b].type( typ )).clone(),
        render_d = (render_d[b].type( typ )).clone(),
        h_real = (h_real[b].type( typ )).clone(), 
        h_render = (h_render[b].type( typ )).clone(),
        h_real_est = h_real_est,
        mp = model_points[0].clone(),
        img = real_img_original[0].clone())
    ran_suc = suc_
    suc = suc_ and suc
    #TODO number for pnp-h-ero is faked
    pnp_h_ero = pnp_h_gt
#     suc_, pnp_h_ero = flow_to_trafo_pnp(
#         real_br = real_br[b].clone(),
#         real_tl = (real_tl[b]).clone(), 
#         ren_br = (ren_br[b]).clone(), 
#         ren_tl = (ren_tl[b]).clone(),
#         flow_mask = (flow_mask_eroded[b]).clone(), 
#         u_map = (u_map[b].type( typ )).clone(),
#         v_map = (v_map[b].type( typ )).clone(), 
#         K_real = (K_real.type( typ )).clone(),
#         K_ren = (K_ren.type( typ )).clone(),
#         real_d = (real_d[b].type( typ )).clone(),
#         render_d = (render_d[b].type( typ )).clone(),
#         h_real = (h_real[b].type( typ )).clone(), 
#         h_render = (h_render[b].type( typ )).clone(),
#         h_real_est = h_real_est,
#         mp = model_points[0].clone() )
#     ran_suc = ran_suc and suc_
    suc = suc_ and suc
    
    
     ### GT Label ###
    suc_, P_real_in_center, P_ren_in_center, P_real_trafo, gt_T_res = flow_to_trafo(
        real_br = real_br[b].clone(),
        real_tl = (real_tl[b]).clone(), 
        ren_br = (ren_br[b]).clone(), 
        ren_tl = (ren_tl[b]).clone(),
        flow_mask = (flow_mask[b]).clone(), 
        u_map = (u_map[b].type( typ )).clone(),
        v_map = (v_map[b].type( typ )).clone(), 
        K_real = (K_real.type( typ )).clone(),
        K_ren = (K_ren.type( typ )).clone(),
        real_d = (real_d[b].type( typ )).clone(),
        render_d = (render_d[b].type( typ )).clone(),
        h_real = (h_real_est.type( typ )).clone(), 
        h_render = (h_render[b].type( typ )).clone())
    gt_h_est =  gt_T_res @ h_render[0].type(typ)
    suc = suc_ and suc
    
    ### Erode Label ###
    
    suc_,_,_,_, ero_T_res = flow_to_trafo(
        real_br = copy.deepcopy(real_br[b]),
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask_eroded[b]), 
        u_map = copy.deepcopy((u_map[b]).type( typ )), 
        v_map = copy.deepcopy((v_map[b]).type( typ )), 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))
    ero_h_est =  ero_T_res @ h_render[0].type(typ) # set rotation
    suc = suc_ and suc
    
    
    ### N1 ###
    lvl = 20
    n1 = torch.rand(v_map[b].shape, dtype = typ)
    n2 = torch.rand(v_map[b].shape, dtype = typ)
    u_map_clone = (u_map[b]).type( typ ).clone() + (n1 - 0.5) * 2 * lvl
    v_map_clone = (v_map[b]).type( typ ).clone() + (n2 - 0.5) * 2 * lvl
    
    suc_,_,_,_, n1_T_res = flow_to_trafo(
        real_br = copy.deepcopy(real_br[b]),
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask_eroded[b]), 
        u_map = u_map_clone, 
        v_map = v_map_clone, 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))
    n1_h_est =  n1_T_res @ h_render[0].type(typ) # set rotation
    suc = suc_ and suc
    ### N2 ###
    lvl = 20
    n1 = float(torch.rand((1), dtype = typ ))
    n2 = float(torch.rand((1), dtype = typ ))
    u_map_clone = (u_map[b]).type( typ ).clone() + float( (n1 - 0.5) * 2 * lvl )
    v_map_clone = (v_map[b]).type( typ ).clone() + float( (n2 - 0.5) * 2 * lvl )
    
    
    suc_,_,_,_, n2_T_res = flow_to_trafo(
        real_br = copy.deepcopy(real_br[b]),
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask_eroded[b]), 
        u_map = u_map_clone, 
        v_map = v_map_clone, 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))
    n2_h_est =  n2_T_res @ h_render[0].type(typ) # set rotation
    suc = suc_ and suc
    
    if not ran_suc:
        failed_desig_ls.append( int( unique_desig[1]) )
        print(f"Ransac failed for {int( unique_desig[1])} {unique_desig[0]}")
        continue
        
        
    mask = (flow_mask == True)
    # Target Model-points Transformed
    p = model_points.shape[1]
    target = torch.bmm( model_points, torch.transpose(h_real[:,:3,:3], 1,2 ) ) + h_real[:,:3,3][:,None,:].repeat(1,p,1)

    # Compute ADD-S
    adds_res_gt_flow = criterion_adds(target[b][None].clone(), model_points[b][None].clone(), idx[b][None], H = gt_h_est[None].type( target.dtype) )
    adds_res_gt_flow_eroded = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = ero_h_est[None].type( target.dtype) )
    adds_res_n1 = criterion_adds(target[b][None].clone(), model_points[b][None].clone(), idx[b][None], H = n1_h_est[None].type( target.dtype) )
    adds_res_n2 = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = n2_h_est[None].type( target.dtype) )
    adds_res_pnp_gt = criterion_adds(target[b][None].clone(), model_points[b][None].clone(), idx[b][None], H = pnp_h_gt[None].type( target.dtype) )
    adds_res_pnp_ero = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = pnp_h_ero[None].type( target.dtype) )
    adds_init = criterion_adds(target[b][None].clone(), model_points[b][None].clone(), idx[b][None], H = h_real_est[None].type( target.dtype))
    
    
    v1,v2,v3 = float(adds_res_gt_flow.detach()),float(adds_init.detach()), float(adds_res_gt_flow_eroded.detach())
    v4,v5,v6,v7 = float(adds_res_n1.detach()),float(adds_res_n2.detach()),float(adds_res_pnp_gt.detach()), float(adds_res_pnp_ero.detach())
    
    test_values = [int( unique_desig[1]),v1,v2,v3,v4,v5,v6,v7]
    res = {names[i]: test_values[i] for i in range(len(names))} 
    df = df.append(res, ignore_index=True)
        
    

    # Append results to list
    desig_ls.append(int( unique_desig[1]) )
    adds_init_ls.append(float(adds_init.detach()))
    adds_gt_ls.append(float(adds_res_gt_flow.detach()))
    adds_erode_ls.append(float(adds_res_gt_flow_eroded.detach()))    
    adds_n1_ls.append(float(adds_res_n1.detach()))
    adds_n2_ls.append(float(adds_res_n2.detach()))
    adds_pnp_gt_ls.append(float(adds_res_pnp_gt.detach()))
    adds_pnp_ero_ls.append(float(adds_res_pnp_ero.detach()))

#     if adds_gt_ls[-1] > 0.02 :
#         print(f'Failed Iteration {j} > 0.02 ADD-S, GT', adds_gt_ls[-1] , 'ERRODED', adds_erode_ls[-1]  )
        
#         no = torch.norm(P_real_trafo- P_ren_in_center, dim=1)
#         print( f'    shape of use points: ',P_real_trafo.shape )
#         print( f'    number of point with a distance greater 0.2 {torch.sum( no>0.2 )} ' )
#         print( f'    number of point with a distance greater 0.02 {torch.sum( no>0.02 )} ' )
#         print( f'    number of point with a distance smaller 0.02 {torch.sum( no<0.02 )} ' )
        
        
#         plot1 = True
#     else:
#         print(f'Suc Iteration {j} < 0.02 ADD-S, GT', adds_gt_ls[-1] , 'ERRODED', adds_erode_ls[-1]  )
#         plot1 = False
#     if idxmax != -1:
#         plot1 = True
        
    
    print(f'Not Iteration {j} < 0.02 ADD-S, GT', adds_gt_ls[-1] , 'ERRODED', adds_erode_ls[-1],'PnP', adds_pnp_gt_ls[-1]   )
        
    if adds_pnp_gt_ls[-1] > 0.02 :
        cou += 1
#         if cou > 5:
#             break
#         continue
#         print("Real depth map cropped")
        real_depth_img = Drawer().disp_img_1d(render_d[b].numpy(),ret=True)
#        real_depth_img = Drawer().disp_img_1d(depth_render_original[0][0].numpy(),ret=True)
#         real_depth_img = np.repeat( real_depth_img[:,:,None],3, axis=2)
#         print("Render depth map cropped")
#         render_depth_img = Drawer().disp_img_1d(render_d[b].numpy(),ret=True)
#         render_depth_img = np.repeat( render_depth_img[:,:,None],3, axis=2)
#         Drawer().disp_img_1d(flow_mask[0],ret=True)

#         sub = max(1,int( P_real_in_center.shape[0]/100 ) )
#         plot_two_pcd_line(P_real_in_center[::sub].numpy(), P_ren_in_center[::sub].numpy() )
#         plot_two_pcd_line(P_real_trafo[::sub].numpy(), P_ren_in_center[::sub].numpy() )

#         print(f"Real Image, Estimated Points given GT Flow {P_real_in_center.shape}")
#         visualizer.plot_estimated_pose( tag = f"_",
#                         epoch = 1,
#                         img= real_img_original[b].cpu().numpy(),
#                         points =copy.deepcopy(model_points[0].cpu().numpy()),
#                         store = False,
#                         jupyter=True,
#                         K = K_real.cpu().numpy(),
#                         H = h_real[0].numpy(),
#                         method='def')
#         print("Real Image Cropped, Estimated Points given GT Flow")
#         visualizer.plot_estimated_pose_on_bb( tag = f"_",
#                         epoch = 1,
#                         img= real_depth_img,
#                         tl = real_tl[0],
#                         br = real_br[0],
#                         points = copy.deepcopy( P_real_in_center.cpu().numpy()),
#                         store = False,
#                         jupyter=True,
#                         K = K_real.cpu().numpy(),
#                         H = np.eye(4),
#                         method='def')

#         fil = label == unique_desig[1]    
#         real_img_original[b][ fil[0][:,:,None].repeat(1,1,3) ] = 255
#         print("Real Image, Estimated Points given GT Flow with label is white")
#         visualizer.plot_estimated_pose( tag = f"_",
#                         epoch = 1,
#                         img= real_img_original[b].cpu().numpy(),
#                         points = copy.deepcopy( P_real_in_center.cpu().numpy()),
#                         store = False,
#                         jupyter=True,
#                         K = K_real.cpu().numpy(),
#                         H = np.eye(4),
#                         method='def')


        
        print("GT: Render Image Cropped")
        visualizer.plot_estimated_pose_on_bb( tag = f"_",
                            epoch = 1,
                            img= render_img[0].numpy(),
                            points = copy.deepcopy(model_points[0].cpu().numpy()),
                            tl = ren_tl[0],
                            br = ren_br[0],
                            store = False,
                            jupyter=True,
                            K = K_ren.cpu().numpy(),
                            H = h_render[0].numpy(),
                            method='def')
        print("PNP, Real Image")
        visualizer.plot_estimated_pose( tag = f"_",
                            epoch = 1,
                            img= real_img_original[b].cpu().numpy(),
                            points = copy.deepcopy(model_points[0].cpu().numpy()),
                            store = False,
                            jupyter= True,
                            K = K_ren.cpu().numpy(),
                            H = pnp_h_gt.clone().numpy(),
                            method='def')
        
        print("PNP: Render Image Cropped")
        visualizer.plot_estimated_pose_on_bb( tag = f"_",
                            epoch = 1,
                            img= render_img[0].numpy(),
                            points = copy.deepcopy(model_points[0].cpu().numpy()),
                            tl = ren_tl[0],
                            br = ren_br[0],
                            store = False,
                            jupyter=True,
                            K = K_ren.cpu().numpy(),
                            H = h_render[0].clone().numpy(),
                            method='def')

#         print('Valid Flow Mask')
#         visualizer.plot_segmentation('tag', 1, flow_mask_eroded[b] , store=False, method='def', jupyter=True)
#         visualizer.plot_segmentation('tag', 1, flow_mask[b]  , store=False, method='def', jupyter=True)

        print("Corrospondence")
        visualizer.plot_corrospondence(tag=f'_',
                                           epoch=0,
                                            u_map=u_map[0], 
                                            v_map=v_map[0], 
                                            flow_mask=flow_mask[0], 
                                            real_img=real_img[0], 
                                            render_img=render_img[0],
                                            store=False,
                                            jupyter=True,
                                            coloful=True,
                                            method='def',
                                            res_h=10,
                                            res_w=10)
        
         
    #if float(adds_res_pnp_gt.detach()) > 0.02 :
     #   break
# print(f'Result, GT Avg', sum(adds_gt_ls)/len(adds_gt_ls), 'Errode Avg', sum(adds_erode_ls)/len(adds_erode_ls) )
print("Finished")



In [ ]:
def pnp(*args, **kwargs):
    real_br = kwargs['real_br']
    real_tl = kwargs['real_tl']
    ren_br = kwargs['ren_br']
    ren_tl = kwargs['ren_tl']
    flow_mask = kwargs['flow_mask']
    u_map = kwargs['u_map']
    v_map = kwargs['v_map']
    K_real = kwargs['K_real']
    K_ren = kwargs['K_ren']
    real_d = kwargs['real_d']
    render_d = kwargs['render_d']
    h_real = kwargs['h_real']
    h_render = kwargs['h_render']
    h_real_est = kwargs['h_real_est']
    mp = kwargs['mp']
    img = kwargs['img']
    
    typ = u_map.dtype

    # Grid for upsampled real
    grid_real_h = torch.linspace(int(real_tl[0]) ,int(real_br[0]) , 480, device=u_map.device)[:,None].repeat(1,640)
    grid_real_w = torch.linspace(int(real_tl[1]) ,int(real_br[1]) , 640, device=u_map.device)[None,:].repeat(480,1)
    # Project depth map to the pointcloud real
    cam_scale = 10000
    real_pixels = torch.stack( [grid_real_w[flow_mask], grid_real_h[flow_mask], torch.ones(grid_real_h.shape, device = u_map.device,  dtype= u_map.dtype)[flow_mask]], dim=1 ).type(typ)
    K_inv = torch.inverse(K_real.type(torch.float64)).type(typ)
    P_real = K_inv @ real_pixels.T
    P_real = P_real.type(torch.float64) * real_d[flow_mask] / cam_scale
    P_real = P_real.T


    real_pixels_fil = real_pixels


    grid_ren_h = torch.linspace(int(ren_tl[0]) ,int(ren_br[0]), 480, device=u_map.device)[:,None].repeat(1,640)
    grid_ren_w = torch.linspace(int(ren_tl[1]) ,int(ren_br[1]) , 640, device=u_map.device)[None,:].repeat(480,1)
    crop_d_pixels = torch.stack( [grid_ren_w.flatten(), grid_ren_h.flatten(), torch.ones(grid_ren_w.shape, device = u_map.device,  dtype= torch.float64).flatten()], dim=1 ).type(typ)
    K_inv = torch.inverse(K_ren.type(torch.float64)).type(typ)
    P_crop_d = K_inv @ crop_d_pixels.T.type(typ)
    P_crop_d = P_crop_d.type(torch.float64) * render_d.flatten() / cam_scale
    P_crop_d = P_crop_d.T


    render_d_ind_h = torch.linspace(0 ,479 , 480, device=u_map.device)[:,None].repeat(1,640)
    render_d_ind_w= torch.linspace(0 ,639 , 640, device=u_map.device)[None,:].repeat(480,1)
    render_d_ind_h = torch.clamp((render_d_ind_h - u_map).type(torch.float64) ,0,479).type( torch.long )[flow_mask]
    render_d_ind_w = torch.clamp((render_d_ind_w - v_map).type(torch.float32),0,639).type( torch.long )[flow_mask] 
    index = render_d_ind_h*640 + render_d_ind_w # hacky indexing along two dimensions

    P_crop_d  = P_crop_d[index] 
    crop_d_pixels = crop_d_pixels[index] 


    m = filter_pcd( P_crop_d)
    P_crop_d  = P_crop_d[ m ]
    P_real = P_real[m]
    real_pixels = real_pixels[m]
    
    P_ren = P_crop_d

    # random shuffel
    # pts_trafo = min(P_real.shape[0], 50000)
    # idx = torch.randperm( P_real.shape[0] )[0:pts_trafo]
    # P_real = P_real[idx]
    # P_ren = P_ren[idx]
    # real_pixels_fil = crop_d_pixels[idx]

    nr = 10000
    mp_render = get_H(mp)
    mp_render = (mp_render.type(typ)@ h_render.type(typ).T)[::5,:3]
    mp_real = get_H(mp)
    mp_real = (mp_real.type(typ) @ h_real.type(typ).T)[::5,:3]

#     visualizer.plot_estimated_pose_on_bb( tag = f"_",
#         epoch = 1,
#         img= render_img[0].numpy()  ,
#         tl = ren_tl,
#         br = ren_br,
#         points = P_ren[:nr,:3],
#         store = False,
#         jupyter= True,
#         K = K_ren.cpu().numpy(),
#         H = np.eye(4),
#         method='def')

#     plot_two_pcd(mp_render.numpy(), P_ren[:nr,:3].numpy())

#     #inp = render_img[0].clone()
#     inp = render_img_original[0].clone()
#     inp = real_img_original[0].clone()
#     for j in range(0,nr):
#         try:

#             v = int(real_pixels[j,0])
#             u = int(real_pixels[j,1])
#             inp[u:u+4,v:v+4,: ] = torch.tensor( [255,0,255])
#         except:
#             pass
#     display( Image.fromarray( np.uint8(inp.numpy())) )

    P_ren_in_origin =  (get_H( P_ren ).type(typ) @ torch.inverse( h_render.type(torch.float64) ).type(typ).T) [:,:3]

    # PNP estimation
    objectPoints = P_ren_in_origin.clone().numpy()    
    imagePoints = real_pixels[:,:2].numpy()
    dist = np.array( [[0.0,0.0,0.0,0.0]] )

    if objectPoints.shape[0] < 8:
        print('Failed due to missing corsspondences')
    # set current guess as the inital estimate

    rvec = R.from_matrix(h_real_est[:3,:3]).as_rotvec().astype(np.float32)
    tvec = h_real_est[:3,3].numpy().astype(np.float32)
    # calculate PnP between the pixels coordinates in the real image and the corrosponding points in the origin frame
    r_vec2, t_vec2 = cv2.solvePnPRefineLM(copy.deepcopy(objectPoints), \
        copy.deepcopy(imagePoints), 
        K_real.numpy(), 
        dist, 
        copy.deepcopy(rvec),
        copy.deepcopy( tvec))
    r_vec2 = r_vec2[:,None]

    h = rvec_tvec_to_H(r_vec2[:,0],t_vec2)
    return True,torch.from_numpy( h).type(u_map.dtype) 


h = pnp(real_br = real_br[b].clone(),
    real_tl = (real_tl[b]).clone() ,
    ren_br = (ren_br[b]).clone(),
    ren_tl = (ren_tl[b]).clone(),
    flow_mask = (flow_mask[b]).clone(),
    u_map = (u_map[b].type( typ )).clone(),
    v_map = (v_map[b].type( typ )).clone(), 
    K_real = (K_real.type( typ )).clone(),
    K_ren = (K_ren.type( typ )).clone(),
    real_d = (real_d[b].type( typ )).clone(),
    render_d = (render_d[b].type( typ )).clone(),
    h_real = (h_real[b].type(typ )).clone(),
    h_render = (h_render[b].type( typ )).clone(),
    h_real_est = h_real_est,
    mp = model_points[0],
    img = real_img_original[0].clone() )
    

In [ ]:
#clones once
typ = torch.float64
real_br = real_br[b].clone()
real_tl = (real_tl[b]).clone() 
ren_br = (ren_br[b]).clone()
ren_tl = (ren_tl[b]).clone()
flow_mask = (flow_mask[b]).clone()
u_map = (u_map[b].type( typ )).clone()
v_map = (v_map[b].type( typ )).clone() 
K_real = (K_real.type( typ )).clone()
K_ren = (K_ren.type( typ )).clone()
real_d = (real_d[b].type( typ )).clone()
render_d = (render_d[b].type( typ )).clone()
h_real = (h_real[b].type(typ )).clone()
h_render = (h_render[b].type( typ )).clone()
h_real_est = h_real_est
mp = model_points[0]
img = real_img_original[0].clone()

In [ ]:
# Grid for upsampled real
grid_real_h = torch.linspace(int(real_tl[0]) ,int(real_br[0]) , 480, device=u_map.device)[:,None].repeat(1,640)
grid_real_w = torch.linspace(int(real_tl[1]) ,int(real_br[1]) , 640, device=u_map.device)[None,:].repeat(480,1)
# Project depth map to the pointcloud real
cam_scale = 10000
real_pixels = torch.stack( [grid_real_w[flow_mask], grid_real_h[flow_mask], torch.ones(grid_real_h.shape, device = u_map.device,  dtype= u_map.dtype)[flow_mask]], dim=1 ).type(u_map.dtype)
K_inv = torch.inverse(K_real.type(torch.float64))
P_real = K_inv @ real_pixels.T.type(torch.float64)
P_real = P_real.type(torch.float64) * real_d[flow_mask] / cam_scale
P_real = P_real.T


real_pixels_fil = real_pixels


grid_ren_h = torch.linspace(int(ren_tl[0]) ,int(ren_br[0]), 480, device=u_map.device)[:,None].repeat(1,640)
grid_ren_w = torch.linspace(int(ren_tl[1]) ,int(ren_br[1]) , 640, device=u_map.device)[None,:].repeat(480,1)
crop_d_pixels = torch.stack( [grid_ren_w.flatten(), grid_ren_h.flatten(), torch.ones(grid_ren_w.shape, device = u_map.device,  dtype= torch.float64).flatten()], dim=1 ).type(torch.float64)
K_inv = torch.inverse(K_ren.type(torch.float64))
P_crop_d = K_inv @ crop_d_pixels.T.type(torch.float64)
P_crop_d = P_crop_d.type(torch.float64) * render_d.flatten() / cam_scale
P_crop_d = P_crop_d.T



render_d_ind_h = torch.linspace(0 ,479 , 480, device=u_map.device)[:,None].repeat(1,640)
render_d_ind_w= torch.linspace(0 ,639 , 640, device=u_map.device)[None,:].repeat(480,1)
render_d_ind_h = torch.clamp((render_d_ind_h - u_map).type(torch.float64) ,0,479).type( torch.long )[flow_mask]
render_d_ind_w = torch.clamp((render_d_ind_w - v_map).type(torch.float32),0,639).type( torch.long )[flow_mask] 
index = render_d_ind_h*640 + render_d_ind_w # hacky indexing along two dimensions

P_crop_d  = P_crop_d[index] 
crop_d_pixels = crop_d_pixels[index] 


m = filter_pcd( P_crop_d)
P_crop_d  = P_crop_d[ m ]
P_real = P_real[m]
real_pixels = real_pixels[m]


P_ren = P_crop_d


print(P_ren.shape, P_real.shape,index.shape)

# random shuffel
# pts_trafo = min(P_real.shape[0], 50000)
# idx = torch.randperm( P_real.shape[0] )[0:pts_trafo]
# P_real = P_real[idx]
# P_ren = P_ren[idx]
# real_pixels_fil = crop_d_pixels[idx]

nr = 10000
mp_render = get_H(mp)
mp_render = (mp_render.type(torch.float64) @ h_render.type(torch.float64).T)[::5,:3]
mp_real = get_H(mp)
mp_real = (mp_real.type(torch.float64) @ h_real.type(torch.float64).T)[::5,:3]

visualizer.plot_estimated_pose_on_bb( tag = f"_",
    epoch = 1,
    img= render_img[0].numpy()  ,
    tl = ren_tl,
    br = ren_br,
    points = P_ren[:nr,:3],
    store = False,
    jupyter= True,
    K = K_ren.cpu().numpy(),
    H = np.eye(4),
    method='def')

plot_two_pcd(mp_render.numpy(), P_ren[:nr,:3].numpy())

#inp = render_img[0].clone()
inp = render_img_original[0].clone()
inp = real_img_original[0].clone()
for j in range(0,nr):
    try:
        
        v = int(real_pixels[j,0])
        u = int(real_pixels[j,1])
        inp[u:u+4,v:v+4,: ] = torch.tensor( [255,0,255])
    except:
        pass
display( Image.fromarray( np.uint8(inp.numpy())) )
        
P_ren_in_origin =  (get_H( P_ren ) @ torch.inverse( h_render ).T) [:,:3]
    
# PNP estimation
objectPoints = P_ren_in_origin.clone().numpy()    
imagePoints = real_pixels[:,:2].numpy()
dist = np.array( [[0.0,0.0,0.0,0.0]] )

if objectPoints.shape[0] < 8:
    print('Failed due to missing corsspondences')
# set current guess as the inital estimate

rvec = R.from_matrix(h_real[:3,:3]).as_rotvec().astype(np.float32)
tvec = h_real[:3,3].numpy().astype(np.float32)
# calculate PnP between the pixels coordinates in the real image and the corrosponding points in the origin frame

print(imagePoints.shape, objectPoints.shape)
r_vec2, t_vec2 = cv2.solvePnPRefineLM(copy.deepcopy(objectPoints), \
    copy.deepcopy(imagePoints), 
    K_real.numpy(), 
    dist, 
    copy.deepcopy(rvec),
    copy.deepcopy( tvec))
r_vec2 = r_vec2[:,None]

# retval, r_vec2, t_vec2, inliers = cv2.solvePnPRansac(copy.deepcopy(objectPoints), \
#     copy.deepcopy(imagePoints), 
#     K_real.numpy(), 
#     dist, 
#     copy.deepcopy(rvec),
#     copy.deepcopy(tvec), 
#     useExtrinsicGuess = True, iterationsCount = 1000, reprojectionError= 0.3 ) 

h = rvec_tvec_to_H(r_vec2[:,0],t_vec2)
print("H predicted \n",h, "\n H real \n", h_real)

P_ren_h_trafo_out = (get_H(P_ren_in_origin) @ h.T)[:,:3]
sub = int(P_ren_h_trafo_out.shape[0]/500 )

plot_two_pcd_line(P_ren_h_trafo_out[::sub,:3].numpy(), mp_real.clone().numpy() )


In [ ]:
# Visualizing all inputs
# plotting
img_np = np.uint8(img.clone().numpy())
visualizer.plot_bounding_box("start", 0, img_np, rmin=int(real_tl[0]), rmax=int(real_br[0]), cmin=int(real_tl[1]), cmax=int(real_br[1]), str_width=2, store=False, jupyter=True)


sub = int(P_ren.shape[0]/200)
mp_render = get_H(mp)
mp_render = (mp_render.type(torch.float64) @ h_render.type(torch.float64).T)[::5,:3]
print("Check alignment betweem rendered pointcloud and the cad model rotated to the pose of the PCD")
plot_two_pcd(P_ren[::sub,:3].numpy(), mp_render.numpy())

# # The same filtering is performed for the PnP as for the Pointcloud matching approach! 
# # Transform the known rendered point corrspondences to the origin frame. 

P_ren_h = get_H(P_ren)
P_ren_h_trafo = P_ren_h.clone() @ torch.inverse(h_render).T


# print("Points render in center. CAD Model canter")
# plot_two_pcd(P_ren_h_trafo[::sub,:3].numpy(), mp[::5].numpy())


mp_real = get_H(mp)
mp_real = (mp_real.type(torch.float64) @ h_real.type(torch.float64).T)[::5,:3]
# print("Real points", "CAD Model at gt position")
# plot_two_pcd(mp_real.numpy(), P_real[::sub].numpy())

# print("Corrospondence")
visualizer.plot_corrospondence(tag=f'_',
           epoch=0,
            u_map=u_map, 
            v_map=v_map, 
            flow_mask=flow_mask, 
            real_img=real_img[0], 
            render_img=render_img[0],
            store=False,
            jupyter=True,
            coloful=True,
            method='def',
            res_h=10,
            res_w=10)

img_depth_full = Drawer().disp_img_1d( depth_render_original[0][0], ret= True)
print(img_depth_full.shape, 'img_depth_full')
img_depth_full = img_depth_full[:,:,None].repeat(3,2)
visualizer.plot_bounding_box("start", 0, img_depth_full, rmin=int(ren_tl[0]), rmax=int(ren_br[0]), cmin=int(ren_tl[1]), cmax=int(ren_br[1]), str_width=2, store=False, jupyter=True)
img_depth_full = Drawer().disp_img_1d( render_d, ret= True)


print( 'Image comp',  torch.min( depth_render_original), torch.max( depth_render_original) )
print( 'Image Render D',  torch.min( render_d), torch.max( render_d) )

full_depth = depth_render_original[0][0]
grid_full_d_h = torch.linspace(0 ,479 , 480, device=u_map.device)[:,None].repeat(1,640)
grid_full_d_w = torch.linspace(0 ,639 , 640, device=u_map.device)[None,:].repeat(480,1)

# Project depth map to the pointcloud real
cam_scale = 10000
print( grid_full_d_w.shape, grid_full_d_h.shape, grid_full_d_w.flatten().shape)
full_d_pixels = torch.stack( [grid_full_d_w.flatten(), grid_full_d_h.flatten(), torch.ones(grid_full_d_w.shape, device = u_map.device,  dtype= torch.float64).flatten()], dim=1 ).type(torch.float64)
print(full_d_pixels.shape)
K_inv = torch.inverse(K_ren.type(torch.float64))
P_full_d = K_inv @ full_d_pixels.T.type(torch.float64)
print(P_full_d.shape, full_depth.flatten().shape)
P_full_d = P_full_d.type(torch.float64) * full_depth.flatten() / cam_scale
P_full_d = P_full_d.T
P_full_d  = P_full_d [ filter_pcd( P_full_d) ]
sub = int(P_full_d.shape[0]/200)

print('Alignment full depth map with model points at rendered pose')
plot_two_pcd(mp_render.numpy(), P_full_d[::sub].numpy())


grid_ren_h = torch.linspace(int(ren_tl[0]) ,int(ren_br[0]), 480, device=u_map.device)[:,None].repeat(1,640)
grid_ren_w = torch.linspace(int(ren_tl[1]) ,int(ren_br[1]) , 640, device=u_map.device)[None,:].repeat(480,1)

crop_d_pixels = torch.stack( [grid_ren_w.flatten(), grid_ren_h.flatten(), torch.ones(grid_ren_w.shape, device = u_map.device,  dtype= torch.float64).flatten()], dim=1 ).type(torch.float64)
K_inv = torch.inverse(K_ren.type(torch.float64))

P_crop_d = K_inv @ crop_d_pixels.T.type(torch.float64)
P_crop_d = P_crop_d.type(torch.float64) * render_d.flatten() / cam_scale
P_crop_d = P_crop_d.T

P_crop_d  = P_crop_d [ filter_pcd( P_crop_d) ]
sub = int(P_crop_d.shape[0]/200)
print('Alignment cropped bounding box with Model points at rendered pose')
plot_two_pcd(mp_render.numpy(), P_crop_d[::sub].numpy())



P_ren = 



# visualizer.plot_estimated_pose( tag = f"_",
#     epoch = 1,
#     img= img_bp,
#     points = P_ren_h_trafo_out[:,:3],
#     store = False,
#     jupyter= True,
#     K = K_real.cpu().numpy(),
#     H = np.eye(4),
#     method='left')



# for p in range(imagePoints.shape[0]):
#     u = min(479,int(imagePoints[p,1]) )
#     v = min(639,int(imagePoints[p,0]) )
#     img_np[u,v, :] = np.array([138,43,226])

# Drawer().disp_img_1d(img, hold=False )

In [ ]:
# PNP estimation
objectPoints = P_ren_h_trafo[:,:3].clone().numpy() #P_ren.numpy()    
imagePoints = real_pixels_fil[:,:2].numpy()
dist = np.array( [[0.0,0.0,0.0,0.0]] )
div = 1 # int(objectPoints.shape[0]/1000)

print(objectPoints.shape)
print(imagePoints.shape)
if objectPoints.shape[0] < 8:
    print('Failed due to missing corsspondences')
# set current guess as the inital estimate

rvec = R.from_matrix(h_real[:3,:3]).as_rotvec().astype(np.float32)
tvec = h_real[:3,3].numpy().astype(np.float32)
# calculate PnP between the pixels coordinates in the real image and the corrosponding points in the origin frame
r_vec2, t_vec2 = cv2.solvePnPRefineLM(copy.deepcopy(objectPoints), \
    copy.deepcopy(imagePoints), 
    K_real.numpy(), 
    dist, 
    copy.deepcopy(rvec),
    copy.deepcopy( tvec))
r_vec2 = r_vec2[:,None]

retval, r_vec2, t_vec2, inliers = cv2.solvePnPRansac(copy.deepcopy(objectPoints), \
    copy.deepcopy(imagePoints), 
    K_real.numpy(), 
    dist, 
    copy.deepcopy(rvec),
    copy.deepcopy(tvec), 
    useExtrinsicGuess = True, iterationsCount = 1000, reprojectionError= 0.3 ) 


         
print(retval, inliers.shape)

h = rvec_tvec_to_H(r_vec2[:,0],t_vec2)
print("H predicted \n",h, "\n H real \n", h_real)
P_ren_h_trafo_out = P_ren_h_trafo @ h.T
sub = int(P_ren_h_trafo_out.shape[0]/500 )
plot_two_pcd_line(P_ren_h_trafo_out[::sub,:3].numpy(), mp_real.clone().numpy() )


In [ ]:
#         print("Real depth map cropped")
flow_mask.shape
#         real_depth_img = Drawer().disp_img_1d(render_d[b].numpy(),ret=True)
lab = Drawer().disp_img_1d(flow_mask.numpy(),ret=True)
#         real_depth_img = np.repeat( real_depth_img[:,:,None],3, axis=2)
#         print("Render depth map cropped")
#         render_depth_img = Drawer().disp_img_1d(render_d[b].numpy(),ret=True)
#         render_depth_img = np.repeat( render_depth_img[:,:,None],3, axis=2)
#         Drawer().disp_img_1d(flow_mask[0],ret=True)
Drawer().disp_img_1d(render_d.numpy())


# plotting

img_np = np.uint8(img.clone().numpy())
for p in range(imagePoints.shape[0]):

    u = min(479,int(imagePoints[p,1]) )
    v = min(639,int(imagePoints[p,0]) )
    img_np[u,v, :] = np.array([138,43,226])

Drawer().disp_img_1d(img, hold=False )
visualizer.plot_estimated_pose( tag = f"_",
    epoch = 1,
    img= img_np,
    points = P_ren_h_trafo_out[:2,:3],
    store = False,
    jupyter= True,
    K = K_real.cpu().numpy(),
    H = np.eye(4),
    method='right')

o = visualizer.plot_estimated_pose( tag = f"_",
    epoch = 1,
    img= img_np,
    points = P_ren_h_trafo_out[:,:3],
    store = False,
    jupyter= True,
    K = K_real.cpu().numpy(),
    H = np.eye(4),
    method='left')


In [ ]:
from visu import plot_two_pcd

import pandas as pd
# idxmax can be set to != -1 to load specific sample

idxmax = -1 
device = 'cpu'
desig_ls= []
adds_gt_ls = []
adds_init_ls = []
adds_erode_ls = []
adds_n1_ls = []
adds_n2_ls = []

adds_pnp_gt_ls = []
adds_pnp_ero_ls = []

failed_desig_ls = []
visualizer = Visualizer('/home/jonfrey/Debug', None)
print('START')

K_ren = torch.tensor( dataset_train._backend.get_camera('data_syn/0019', K=True), device=device ) 
K_ren = torch.tensor( dataset_train._backend.get_camera('data/0049', K=True), device=device ) 

max_iter = 10

if idxmax != -1:
    print(idxmax, "is not equal to -1 load batch")
    batch = next(itertools.islice(dataloader_train, idxmax, None))[0]
cou = 0
print("START")
names = ['ID','ADD-S GT','ADD-S INITAL','ADD-S Eroded Mask', 'ADD-S Random', 'ADD-S Bias', 'ADD-S PNP GT', 'ADD-S PNP Ero']
df = pd.DataFrame(columns=names)
        
for j,ba in enumerate(dataloader_test):
    
    if j > max_iter-1:
        break
    if j % 10 == 0:
        print(f"Processed {j}/{max_iter}")

    if idxmax == -1:  
        batch = ba[0]
    # use first sample in batch 
    b = 0
    
    model_points = batch[4]
    idx = batch[5]  # Be carefull here the first objects starts with 0. Normally 0 is the NO object class in all other datastructures
    label = batch[7]
    real_img_original = batch[8]
    cam = batch[9]
    gt_rot_wxyz, gt_trans, unique_desig = batch[10:13] # unique_desig[1] contains the idx starting at 1 for the first object 
    bs = model_points.shape[0]
    if batch[13] is False:
        print('Continue')
        continue
    real_img, render_img, real_d, render_d, gt_label_cropped = batch[13:18]
    pred_rot_wxyz, pred_trans, pred_points, h_render, h_real, render_img_original = batch[18:24]
    u_map, v_map, flow_mask, bb, depth_render_original= batch[24:]
    real_tl, real_br, ren_tl, ren_br = bb 
    
    data = torch.cat([real_img, render_img], dim=1)
    uv_gt = torch.stack( [u_map, v_map], dim=3 ).permute(0,3,1,2)
    
    
    ero_in = (gt_label_cropped ==  unique_desig[1])[:,None,:,:].type(torch.float32) # BS,C,H,W
    t_size = get_scale_for_erosion(ero_in)
    ero_out = eroision_batch(ero_in,t_size)
    
    # get camera
    K_real = torch.tensor( [[cam[b,2],0,cam[b,0]],[b,cam[b,3],cam[b,1]],[0,0,1]], device=device )
    
    #get inital estimate of the poistion given by the dataloader
    h_real_est = torch.eye(4,device=device)
    h_real_est[:3,:3] = quat_to_rot(pred_rot_wxyz[b][None,:], conv='wxyz', device=device)
    h_real_est[:3,3] = torch.tensor( pred_trans[b].clone().detach() ,device=device )
    
    #print(h_render)
    #break
    
    ### GT Estimate ###
    typ = u_map.dtype
    fmt = flow_mask.dtype
    flow_mask_eroded  = (flow_mask * ero_out.type(torch.float32)).type(fmt)[:,0]
    suc = True
    suc_, pnp_h_gt = flow_to_trafo_pnp(
        real_br = real_br[b].clone(),
        real_tl = (real_tl[b]).clone(), 
        ren_br = (ren_br[b]).clone(), 
        ren_tl = (ren_tl[b]).clone(),
        flow_mask = (flow_mask[b]).clone(), 
        u_map = (u_map[b].type( typ )).clone(),
        v_map = (v_map[b].type( typ )).clone(), 
        K_real = (K_real.type( typ )).clone(),
        K_ren = (K_ren.type( typ )).clone(),
        real_d = (real_d[b].type( typ )).clone(),
        render_d = (render_d[b].type( typ )).clone(),
        h_real = (h_real[b].type( typ )).clone(), 
        h_render = (h_render[b].type( typ )).clone(),
        h_real_est = h_real_est,
        mp = model_points[0].clone(),
        img = real_img_original[0].clone())
    ran_suc = suc_
    suc = suc_ and suc
    #TODO number for pnp-h-ero is faked
    pnp_h_ero = pnp_h_gt
#     suc_, pnp_h_ero = flow_to_trafo_pnp(
#         real_br = real_br[b].clone(),
#         real_tl = (real_tl[b]).clone(), 
#         ren_br = (ren_br[b]).clone(), 
#         ren_tl = (ren_tl[b]).clone(),
#         flow_mask = (flow_mask_eroded[b]).clone(), 
#         u_map = (u_map[b].type( typ )).clone(),
#         v_map = (v_map[b].type( typ )).clone(), 
#         K_real = (K_real.type( typ )).clone(),
#         K_ren = (K_ren.type( typ )).clone(),
#         real_d = (real_d[b].type( typ )).clone(),
#         render_d = (render_d[b].type( typ )).clone(),
#         h_real = (h_real[b].type( typ )).clone(), 
#         h_render = (h_render[b].type( typ )).clone(),
#         h_real_est = h_real_est,
#         mp = model_points[0].clone() )
#     ran_suc = ran_suc and suc_
    suc = suc_ and suc
    
    
     ### GT Label ###
    suc_, P_real_in_center, P_ren_in_center, P_real_trafo, gt_T_res = flow_to_trafo(
        real_br = real_br[b].clone(),
        real_tl = (real_tl[b]).clone(), 
        ren_br = (ren_br[b]).clone(), 
        ren_tl = (ren_tl[b]).clone(),
        flow_mask = (flow_mask[b]).clone(), 
        u_map = (u_map[b].type( typ )).clone(),
        v_map = (v_map[b].type( typ )).clone(), 
        K_real = (K_real.type( typ )).clone(),
        K_ren = (K_ren.type( typ )).clone(),
        real_d = (real_d[b].type( typ )).clone(),
        render_d = (render_d[b].type( typ )).clone(),
        h_real = (h_real_est.type( typ )).clone(), 
        h_render = (h_render[b].type( typ )).clone())
    gt_h_est =  gt_T_res @ h_render[0].type(typ)
    suc = suc_ and suc
    
    ### Erode Label ###
    
    suc_,_,_,_, ero_T_res = flow_to_trafo(
        real_br = copy.deepcopy(real_br[b]),
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask_eroded[b]), 
        u_map = copy.deepcopy((u_map[b]).type( typ )), 
        v_map = copy.deepcopy((v_map[b]).type( typ )), 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))
    ero_h_est =  ero_T_res @ h_render[0].type(typ) # set rotation
    suc = suc_ and suc
    
    
    ### N1 ###
    lvl = 20
    n1 = torch.rand(v_map[b].shape, dtype = typ)
    n2 = torch.rand(v_map[b].shape, dtype = typ)
    u_map_clone = (u_map[b]).type( typ ).clone() + (n1 - 0.5) * 2 * lvl
    v_map_clone = (v_map[b]).type( typ ).clone() + (n2 - 0.5) * 2 * lvl
    
    suc_,_,_,_, n1_T_res = flow_to_trafo(
        real_br = copy.deepcopy(real_br[b]),
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask_eroded[b]), 
        u_map = u_map_clone, 
        v_map = v_map_clone, 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))
    n1_h_est =  n1_T_res @ h_render[0].type(typ) # set rotation
    suc = suc_ and suc
    ### N2 ###
    lvl = 20
    n1 = float(torch.rand((1), dtype = typ ))
    n2 = float(torch.rand((1), dtype = typ ))
    u_map_clone = (u_map[b]).type( typ ).clone() + float( (n1 - 0.5) * 2 * lvl )
    v_map_clone = (v_map[b]).type( typ ).clone() + float( (n2 - 0.5) * 2 * lvl )
    
    
    suc_,_,_,_, n2_T_res = flow_to_trafo(
        real_br = copy.deepcopy(real_br[b]),
        real_tl = copy.deepcopy(real_tl[b]), 
        ren_br = copy.deepcopy(ren_br[b]), 
        ren_tl = copy.deepcopy(ren_tl[b]),
        flow_mask = copy.deepcopy(flow_mask_eroded[b]), 
        u_map = u_map_clone, 
        v_map = v_map_clone, 
        K_real = copy.deepcopy(K_real.type( typ )),
        K_ren = copy.deepcopy(K_ren.type( typ )),
        real_d = copy.deepcopy(real_d[b].type( typ )),
        render_d = copy.deepcopy(render_d[b].type( typ )),
        h_real = copy.deepcopy(h_real_est.type( typ )), 
        h_render = copy.deepcopy(h_render[b].type( typ )))
    n2_h_est =  n2_T_res @ h_render[0].type(typ) # set rotation
    suc = suc_ and suc
    
    if not ran_suc:
        failed_desig_ls.append( int( unique_desig[1]) )
        print(f"Ransac failed for {int( unique_desig[1])} {unique_desig[0]}")
        continue
        
        
    mask = (flow_mask == True)
    # Target Model-points Transformed
    p = model_points.shape[1]
    target = torch.bmm( model_points, torch.transpose(h_real[:,:3,:3], 1,2 ) ) + h_real[:,:3,3][:,None,:].repeat(1,p,1)

    # Compute ADD-S
    adds_res_gt_flow = criterion_adds(target[b][None].clone(), model_points[b][None].clone(), idx[b][None], H = gt_h_est[None].type( target.dtype) )
    adds_res_gt_flow_eroded = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = ero_h_est[None].type( target.dtype) )
    adds_res_n1 = criterion_adds(target[b][None].clone(), model_points[b][None].clone(), idx[b][None], H = n1_h_est[None].type( target.dtype) )
    adds_res_n2 = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = n2_h_est[None].type( target.dtype) )
    adds_res_pnp_gt = criterion_adds(target[b][None].clone(), model_points[b][None].clone(), idx[b][None], H = pnp_h_gt[None].type( target.dtype) )
    adds_res_pnp_ero = criterion_adds(target[b][None], model_points[b][None], idx[b][None], H = pnp_h_ero[None].type( target.dtype) )
    adds_init = criterion_adds(target[b][None].clone(), model_points[b][None].clone(), idx[b][None], H = h_real_est[None].type( target.dtype))
    
    
    v1,v2,v3 = float(adds_res_gt_flow.detach()),float(adds_init.detach()), float(adds_res_gt_flow_eroded.detach())
    v4,v5,v6,v7 = float(adds_res_n1.detach()),float(adds_res_n2.detach()),float(adds_res_pnp_gt.detach()), float(adds_res_pnp_ero.detach())
    
    test_values = [int( unique_desig[1]),v1,v2,v3,v4,v5,v6,v7]
    res = {names[i]: test_values[i] for i in range(len(names))} 
    df = df.append(res, ignore_index=True)
        
    

    # Append results to list
    desig_ls.append(int( unique_desig[1]) )
    adds_init_ls.append(float(adds_init.detach()))
    adds_gt_ls.append(float(adds_res_gt_flow.detach()))
    adds_erode_ls.append(float(adds_res_gt_flow_eroded.detach()))    
    adds_n1_ls.append(float(adds_res_n1.detach()))
    adds_n2_ls.append(float(adds_res_n2.detach()))
    adds_pnp_gt_ls.append(float(adds_res_pnp_gt.detach()))
    adds_pnp_ero_ls.append(float(adds_res_pnp_ero.detach()))

#     if adds_gt_ls[-1] > 0.02 :
#         print(f'Failed Iteration {j} > 0.02 ADD-S, GT', adds_gt_ls[-1] , 'ERRODED', adds_erode_ls[-1]  )
        
#         no = torch.norm(P_real_trafo- P_ren_in_center, dim=1)
#         print( f'    shape of use points: ',P_real_trafo.shape )
#         print( f'    number of point with a distance greater 0.2 {torch.sum( no>0.2 )} ' )
#         print( f'    number of point with a distance greater 0.02 {torch.sum( no>0.02 )} ' )
#         print( f'    number of point with a distance smaller 0.02 {torch.sum( no<0.02 )} ' )
        
        
#         plot1 = True
#     else:
#         print(f'Suc Iteration {j} < 0.02 ADD-S, GT', adds_gt_ls[-1] , 'ERRODED', adds_erode_ls[-1]  )
#         plot1 = False
#     if idxmax != -1:
#         plot1 = True
        
    
    print(f'Not Iteration {j} < 0.02 ADD-S, GT', adds_gt_ls[-1] , 'ERRODED', adds_erode_ls[-1],'PnP', adds_pnp_gt_ls[-1]   )
        
    if adds_pnp_gt_ls[-1] > 0.000005 :
        cou += 1
#         if cou > 5:
#             break
#         continue
#         print("Real depth map cropped")
        real_depth_img = Drawer().disp_img_1d(render_d[b].numpy(),ret=True)
#        real_depth_img = Drawer().disp_img_1d(depth_render_original[0][0].numpy(),ret=True)
#         real_depth_img = np.repeat( real_depth_img[:,:,None],3, axis=2)
#         print("Render depth map cropped")
#         render_depth_img = Drawer().disp_img_1d(render_d[b].numpy(),ret=True)
#         render_depth_img = np.repeat( render_depth_img[:,:,None],3, axis=2)
#         Drawer().disp_img_1d(flow_mask[0],ret=True)

#         sub = max(1,int( P_real_in_center.shape[0]/100 ) )
#         plot_two_pcd_line(P_real_in_center[::sub].numpy(), P_ren_in_center[::sub].numpy() )
#         plot_two_pcd_line(P_real_trafo[::sub].numpy(), P_ren_in_center[::sub].numpy() )

#         print(f"Real Image, Estimated Points given GT Flow {P_real_in_center.shape}")
#         visualizer.plot_estimated_pose( tag = f"_",
#                         epoch = 1,
#                         img= real_img_original[b].cpu().numpy(),
#                         points =copy.deepcopy(model_points[0].cpu().numpy()),
#                         store = False,
#                         jupyter=True,
#                         K = K_real.cpu().numpy(),
#                         H = h_real[0].numpy(),
#                         method='def')
#         print("Real Image Cropped, Estimated Points given GT Flow")
#         visualizer.plot_estimated_pose_on_bb( tag = f"_",
#                         epoch = 1,
#                         img= real_depth_img,
#                         tl = real_tl[0],
#                         br = real_br[0],
#                         points = copy.deepcopy( P_real_in_center.cpu().numpy()),
#                         store = False,
#                         jupyter=True,
#                         K = K_real.cpu().numpy(),
#                         H = np.eye(4),
#                         method='def')

#         fil = label == unique_desig[1]    
#         real_img_original[b][ fil[0][:,:,None].repeat(1,1,3) ] = 255
#         print("Real Image, Estimated Points given GT Flow with label is white")
#         visualizer.plot_estimated_pose( tag = f"_",
#                         epoch = 1,
#                         img= real_img_original[b].cpu().numpy(),
#                         points = copy.deepcopy( P_real_in_center.cpu().numpy()),
#                         store = False,
#                         jupyter=True,
#                         K = K_real.cpu().numpy(),
#                         H = np.eye(4),
#                         method='def')


        
        print("GT: Render Image Cropped")
        visualizer.plot_estimated_pose_on_bb( tag = f"_",
                            epoch = 1,
                            img= render_img[0].numpy(),
                            points = copy.deepcopy(model_points[0].cpu().numpy()),
                            tl = ren_tl[0],
                            br = ren_br[0],
                            store = False,
                            jupyter=True,
                            K = K_ren.cpu().numpy(),
                            H = h_render[0].numpy(),
                            method='def')
        print("PNP, Real Image")
        visualizer.plot_estimated_pose( tag = f"_",
                            epoch = 1,
                            img= real_img_original[b].cpu().numpy(),
                            points = copy.deepcopy(model_points[0].cpu().numpy()),
                            store = False,
                            jupyter= True,
                            K = K_ren.cpu().numpy(),
                            H = pnp_h_gt.clone().numpy(),
                            method='def')
        
        print("PNP: Render Image Cropped")
        visualizer.plot_estimated_pose_on_bb( tag = f"_",
                            epoch = 1,
                            img= render_img[0].numpy(),
                            points = copy.deepcopy(model_points[0].cpu().numpy()),
                            tl = ren_tl[0],
                            br = ren_br[0],
                            store = False,
                            jupyter=True,
                            K = K_ren.cpu().numpy(),
                            H = h_render[0].clone().numpy(),
                            method='def')

#         print('Valid Flow Mask')
#         visualizer.plot_segmentation('tag', 1, flow_mask_eroded[b] , store=False, method='def', jupyter=True)
#         visualizer.plot_segmentation('tag', 1, flow_mask[b]  , store=False, method='def', jupyter=True)

        print("Corrospondence")
        visualizer.plot_corrospondence(tag=f'_',
                                           epoch=0,
                                            u_map=u_map[0], 
                                            v_map=v_map[0], 
                                            flow_mask=flow_mask[0], 
                                            real_img=real_img[0], 
                                            render_img=render_img[0],
                                            store=False,
                                            jupyter=True,
                                            coloful=True,
                                            method='def',
                                            res_h=10,
                                            res_w=10)
        
         
    if float(adds_res_pnp_gt.detach()) > 0.02 :
        break
# print(f'Result, GT Avg', sum(adds_gt_ls)/len(adds_gt_ls), 'Errode Avg', sum(adds_erode_ls)/len(adds_erode_ls) )
print("Finished")



In [ ]:
K_real,K_ren

In [ ]:

################### CROPED #########################################
ren_tl, ren_br
grid_real_h = torch.linspace(int(ren_tl[0,0]) ,int(ren_br[0,0]), 480)[:,None].repeat(1,640)
grid_real_w = torch.linspace(int(ren_tl[0,1]) ,int(ren_br[0,1]), 640)[None,:].repeat(480,1)

# Project depth map to the pointcloud real
cam_scale = 10000

ren_or_pixels = torch.stack( [grid_real_w, grid_real_h, torch.ones(grid_real_h.shape, device = u_map.device,  dtype= u_map.dtype)], dim=2 ).type(u_map.dtype)
K_inv = torch.inverse(K_ren.type(torch.float32)).type(u_map.dtype)
P = K_inv @ ren_or_pixels.view(( -1,3)).T

P = torch.mul(P, render_d[0].view(( -1,1)).repeat((1,3)).T ) / cam_scale
P = P.T

# Transform the known rendered point corrspondences to the origin frame. 

P_ren_h = get_H(P)
P_ren_h = P_ren_h [ P_ren_h[:,2] > 0.05 ]

P_ren_h_trafo = P_ren_h @ torch.inverse(h_render[0]).T
m1 = P_ren_h_trafo[:,:3] < 0.1
m1 = torch.sum(m1.type(torch.float32), dim=1 ) > 2
P_ren_h_trafo2 = P_ren_h_trafo[m1]

sub =int( P_ren_h_trafo2.shape[0]/1000 )
plot_two_pcd_line(P_ren_h_trafo2[::sub,:3].numpy(), model_points[0,:,:].clone().numpy() )
                    
############## ORIG##########################
                           
grid_real_h = torch.linspace(0, 479, 480)[:,None].repeat(1,640)
grid_real_w = torch.linspace(0, 639, 640)[None,:].repeat(480,1)

# Project depth map to the pointcloud real
cam_scale = 10000

ren_or_pixels = torch.stack( [grid_real_w, grid_real_h, torch.ones(grid_real_h.shape, device = u_map.device,  dtype= u_map.dtype)], dim=2 ).type(u_map.dtype)
K_inv = torch.inverse(K_ren.type(torch.float32)).type(u_map.dtype)
P = K_inv @ ren_or_pixels.view(( -1,3)).T

P = torch.mul(P, depth_render_original[0][0].view(( -1,1)).repeat((1,3)).T ) / cam_scale
P = P.T

# Transform the known rendered point corrspondences to the origin frame. 

P_ren_h = get_H(P)
P_ren_h = P_ren_h [ P_ren_h[:,2] > 0.05 ]

P_ren_h_trafo = P_ren_h @ torch.inverse(h_render[0]).T
m1 = P_ren_h_trafo[:,:3] < 0.1
m1 = torch.sum(m1.type(torch.float32), dim=1 ) > 2
P_ren_h_trafo2 = P_ren_h_trafo[m1]

sub =int( P_ren_h_trafo2.shape[0]/1000 )
plot_two_pcd_line(P_ren_h_trafo2[::sub,:3].numpy(), model_points[0,:,:].clone().numpy() )




In [ ]:
h_real[0]

In [ ]:
import pandas

#data = np.array( [desig_ls,adds_gt_ls,adds_init_ls,adds_erode_ls, adds_n1_ls, adds_n2_ls, adds_pnp_gt_ls, adds_pnp_ero_ls] ).T
df_failed = pandas.DataFrame( np.array( [failed_desig_ls] ), columns = ['ID_Failed']) 

#df = pandas.DataFrame( data, columns = ['ID','ADD-S GT','ADD-S INITAL','ADD-S Eroded Mask', 'ADD-S Random', 'ADD-S Bias', 'ADD-S PNP GT', 'ADD-S PNP Ero'] )
#df

In [ ]:
print('Random: Takeing Ground Truth Flow and adding noise uniform in between +20 -20 pixels for each pixel')
print('Bias: Takeing Ground Truth Flow and offsetting the flow by a random value in between +20 -20 pixels for the full image')
print( df['ID'].value_counts() )
# print( df_failed['ID_Failed'].value_counts())
df.groupby(['ID']).mean()



In [ ]:
print("MEAN:\n", df.mean() )
print("\n \nSTD:\n", df.std() )
print("\n \nMAX:\n", df.max())
print("\n \nIDXMAX:\n", df.idxmax())

idxmax = df.idxmax()[0]
print(f'\n \nWorst Sample Index is {idxmax}')

In [ ]:
ls = adds_erode_ls
n_bins = 100
failures_filter = 0.1
fig, axs = plt.subplots(3, 1, sharey=True, tight_layout=True)
axs[0].hist(adds_init_ls, bins=n_bins)

adds_gt_f = [a for a in adds_gt_ls if a < failures_filter]
adds_erode_f = [a for a in adds_erode_ls if a < failures_filter]

axs[1].hist(adds_gt_f, bins=n_bins)
axs[2].hist(adds_erode_f, bins=n_bins)
fig.show()


In [ ]:
conda install cvxpnpl

In [ ]:

import numpy as np
from cvxpnpl import pnp

# fix seed to allow for reproducible results
np.random.seed(0)
np.random.seed(42)

# instantiate a couple of points centered around the origin
pts = 0.6 * (np.random.random((6, 3)) - 0.5)

# Made up projective matrix
K = np.array([[160, 0, 320], [0, 120, 240], [0, 0, 1]])

# A pose
R_gt = np.array(
    [
        [-0.48048015, 0.1391384, -0.86589799],
        [-0.0333282, -0.98951829, -0.14050899],
        [-0.8763721, -0.03865296, 0.48008113],
    ]
)
t_gt = np.array([-0.10266772, 0.25450789, 1.70391109])

# Project points to 2D
pts_2d = (pts @ R_gt.T + t_gt) @ K.T
pts_2d = (pts_2d / pts_2d[:, -1, None])[:, :-1]

# Compute pose candidates. the problem is not minimal so only one
# will be provided
poses = pnp(pts_2d=pts_2d, pts_3d=pts, K=K)
R, t = poses[0]

print("Nr of possible poses:", len(poses))
print("R (ground truth):", R_gt, "R (estimate):", R, sep="\n")
print("t (ground truth):", t_gt)
print("t (estimate):", t)